# CLUSTERING WITH KMEANS

- The objective is to find k clusters of similar documents given a certain corpus
- The main analysis are:
 - Find the texts closest to the center of each cluster and manually read them for insight
 - Measure the density of each cluster and how far apart they are from one another
 - The distribution of texts in each class (if the texts are already classified)

In [1]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.cluster import homogeneity_score

from scipy.spatial.distance import cosine as f_dist

import numpy as np

In [2]:
def cluster_items(
    X: np.array, items: list, y : list = [], n_clusters: int = 10, to_scale: bool = True
):
    """
    Returns a list of tuples (item, number of similar items)
    from a X matrix and a list of items
    """
    if to_scale:
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    labels_dict = Counter(kmeans_model.labels_)
    if len(y):
        hom_score = homogeneity_score(y, kmeans_model.labels_)
    else:
        hom_score = -1
    total = len(X)
    closest, _ = pairwise_distances_argmin_min(kmeans_model.cluster_centers_, X)
    return [(items[i], "{:.2f}%".format(100*(labels_dict[kmeans_model.labels_[i]]/total))) for i in set(closest)], hom_score

In [3]:
def statistics_arrays(X, Y):
    rows = []
    for row_x in X:
        for row_y in Y:
            value = 1 - f_dist(row_x, row_y)
            rows.append(value)
    mean_sim = np.mean(rows)
    std_sim = np.std(rows)
    results = {
        "mean_sim_vectors": mean_sim,
        "lower_bound_sim": mean_sim - (2 * std_sim),
        "upper_bound_sim": mean_sim + (2 * std_sim),
    }
    return results

### LOAD DATA

In [4]:
import pandas as pd
import re
from pathlib import Path

df = pd.read_csv(Path().absolute().parent.parent / "data/text_class_8k.csv")

In [5]:
new_rows = []
for row in df.to_dict("records"):
    if re.search(r"alienação fiduciária", row["text"], flags=re.I|re.S):
        new_rows.append(row)
df = pd.DataFrame(new_rows)

In [6]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path().absolute().parent.parent))

from src.text_vectorization import hashing_texts

X = hashing_texts(df["text"], 2**14)

In [7]:
import numpy as np

np.shape(X)

(268, 16384)

## EXPLORATORY ANALYSIS

In [11]:
%%time
list_main_texts_freq, hom_score = cluster_items(X, df["text"].tolist())

Wall time: 2.32 s


In [12]:
hom_score

-1

In [13]:
list_main_texts_freq

[('N. 0710981-54.2019.8.07.0020 - BUSCA E APREENSÃO EM ALIENAÇÃO FIDUCIÁRIA - A: AYMORE CREDITO, FINANCIAMENTO E INVESTIMENTO S.A.. Adv(s).: DF0034392S - MARCO ANTONIO CRESPO BARBOSA. R: ADEMILDE COSTA DE MESQUITA. Adv(s).: Nao Consta Advogado. Poder Judiciário da União TRIBUNAL DE JUSTIÇA DO DISTRITO FEDERAL E DOS TERRITÓRIOS 3VACIVAGCL 3ª Vara Cível de Águas Claras Número do processo: 0710981-54.2019.8.07.0020 Classe judicial: BUSCA E APREENSÃO EM ALIENAÇÃO FIDUCIÁRIA (81) AUTOR: AYMORE CREDITO, FINANCIAMENTO E INVESTIMENTO S.A. RÉU: ADEMILDE COSTA DE MESQUITA SENTENÇA Homologo o pedido de desistência formulado pela parte autora e extingo o processo, sem resolução de mérito, nos termos do inciso VIII do artigo 485 do CPC. Eventuais custas finais pela parte autora. Sem honorários advocatícios, em razão de a relação processual não ter se aperfeiçoado. Publicada esta sentença, independente de certidão emitida pela secretaria, fica desde já certificado o trânsito em julgado, diante da au